In [73]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
import pandas as pd
import os
from tensorflow.python.keras.layers import Input,Dense,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,AveragePooling2D
from tensorflow.python.keras.applications import InceptionV3
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.inception_v3 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import initializers
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.models import Model
import tensorflow as tf

In [162]:
def conv2d_bn(x, nb_filter, num_row, num_col,
              padding='same', strides=(1, 1), use_bias=False):
    
    x = Conv2D(nb_filter, (num_row, num_col),
                      strides=strides,
                      padding=padding,
                      use_bias=use_bias,
                      kernel_regularizer=regularizers.l2(0.00004),
                      kernel_initializer=initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None))(x)
    x = BatchNormalization(axis=-1, momentum=0.9997, scale=False)(x)
    x = Activation('relu')(x)
    
    return x

In [194]:
def stem(x,num_init_channel=32):
    
    conv1_output = conv2d_bn(x, num_init_channel, 3, 3,strides=(2,2))
    branch_0 = conv2d_bn(conv1_output, int(num_init_channel/2), 1, 1)
    branch_0 = conv2d_bn(branch_0, num_init_channel, 3, 3,strides=(2,2))
    print("the left branch is calculated")
    branch_1 = MaxPooling2D(2, strides=(2,2))(conv1_output)
    print("the right branch is calulated")
    out = concatenate([branch_0, branch_1], axis=-1)
    output = conv2d_bn(out, num_init_channel, 1, 1)
    return output

In [195]:
def dense_block(x, num_block, bottleneck_width, k=32):
    
    inter_channel = k*bottleneck_width
    output=x
    
    for index in range(num_block):
        #left channel     
        conv_branch_0 = conv2d_bn(output, inter_channel, 1, 1)
        conv_branch_0 = conv2d_bn(conv_branch_0, k, 3, 3)
        # right channel
        conv_branch_1 = conv2d_bn(output, inter_channel, 1, 1)
        conv_branch_1 = conv2d_bn(conv_branch_1, k, 3, 3)
        conv_branch_1 = conv2d_bn(conv_branch_1, k, 3, 3)

        output = concatenate([conv_branch_0, conv_branch_1], axis=-1)

    return output

In [196]:
def transition_block(x, output_channel, is_avgpool=True):
    
    conv0 = conv2d_bn(x, output_channel, 1, 1, strides=(2,2))
    if is_avgpool:
        output=AveragePooling2D((2,2),strides=(2,2), padding='valid')(conv0)
    else:
        output=conv0
    return output

In [197]:
def classification_layer(x, num_class, keep_prob=0.5):
    
    out = GlobalAveragePooling2D()(x)
    out = Dropout(keep_prob)(out)
    out = Flatten()(out)
    output = Dense(units=num_class, activation='softmax')(out)
    return output

In [198]:
def peleenet(height, width,num_init_channel=3, k=32, block_config=[3,4,8,6], bottleneck_width=[2,2,4,4],classes=10):
    
    inputShape = (height, width, num_init_channel)
    inputs = Input(shape=inputShape)
    x=stem(inputs,num_init_channel)
    x=dense_block(x,block_config[0], bottleneck_width[0],k)
    print(x.shape)
    x=transition_block(x, x.shape[-1], is_avgpool=True)
    x=dense_block(x,block_config[1], bottleneck_width[1],k)
    x=transition_block(x, x.shape[-1], is_avgpool=True)
    x=dense_block(x,block_config[2],bottleneck_width[0],k)
    x=transition_block(x, x.shape[-1], is_avgpool=True)
    x=dense_block(x,block_config[3],bottleneck_width[0],k)
    x=transition_block(x, x.shape[-1], is_avgpool=False)
    x=classification_layer(x, classes, keep_prob=0.5)
    
    model = Model(inputs, x, name="peleenet")
    
    return model

In [199]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# there seems to be an issue with TensorFlow 2.0 throwing non-critical
# warnings regarding gradients when using the model sub-classing
# feature -- I found that by setting the logging level I can suppress
# the warnings from showing up (likely won't be required in future
# releases of TensorFlow)
import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
 
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [200]:
# # initialize the initial learning rate, batch size, and number of
# # epochs to train for
# INIT_LR = 1e-2
# BATCH_SIZE = 128
# NUM_EPOCHS = 60

# # initialize the label names for the CIFAR-10 dataset
# labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog","frog", "horse", "ship", "truck"]

# # load the CIFAR-10 dataset
# print("[INFO] loading CIFAR-10 dataset...")
# ((trainX, trainY), (testX, testY)) = cifar10.load_data()

# # scale the data to the range [0, 1]
# trainX = trainX.astype("float32") / 255.0
# testX = testX.astype("float32") / 255.0

# # convert the labels from integers to vectors
# lb = LabelBinarizer()
# trainY = lb.fit_transform(trainY)
# testY = lb.transform(testY)

# # construct the image generator for data augmentation
# aug = ImageDataGenerator(rotation_range=18, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

[INFO] loading CIFAR-10 dataset...


In [202]:
# model = peleenet(32, 32, num_init_channel=3,k=32, block_config=[1,0,0,0], bottleneck_width=[2,2,4,4], classes=len(labelNames))
# opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
# print("[INFO] training network...")
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# # train the network
# H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),validation_data=(testX, testY),steps_per_epoch=trainX.shape[0] // BATCH_SIZE,epochs=NUM_EPOCHS,verbose=1)
# print("[INFO] evaluating network...")
# predictions = model.predict(testX, batch_size=BATCH_SIZE)
# print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=labelNames))
 
# # determine the number of epochs and then construct the plot title
# N = np.arange(0, NUM_EPOCHS)
# title = "Training Loss and Accuracy on CIFAR-10"

# # plot the training loss and accuracy
# plt.style.use("ggplot")
# plt.figure()
# plt.plot(N, H.history["loss"], label="train_loss")
# plt.plot(N, H.history["val_loss"], label="val_loss")
# plt.plot(N, H.history["accuracy"], label="train_acc")
# plt.plot(N, H.history["val_accuracy"], label="val_acc")
# plt.title(title)
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy")
# plt.legend()
# plt.savefig(args["plot"])

the left branch is calculated
the right branch is calulated
(None, 8, 8, 64)


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'average_pooling2d_13/AvgPool' (op: 'AvgPool') with input shapes: [?,1,1,64].